In [12]:
from music21 import converter, instrument, note, chord
import glob
notes = []

for file in glob.glob("data/*.mid"):
    midi = converter.parse(file)
    notes_to_parse = None

    parts = instrument.partitionByInstrument(midi)

    if parts: # file has instrument parts
        notes_to_parse = parts.parts[0].recurse()
    else: # file has notes in a flat structure
        notes_to_parse = midi.flat.notes

    for element in notes_to_parse:
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            notes.append('.'.join(str(n) for n in element.normalOrder))

In [13]:
import numpy
from keras.utils import np_utils
sequence_length = 30

# get all pitch names
pitchnames = sorted(set(item for item in notes))

# create a dictionary to map pitches to integers
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

network_input = []
network_output = []

# create input sequences and the corresponding outputs
for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i:i + sequence_length]
    sequence_out = notes[i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])

n_patterns = len(network_input)
n_vocab=93
# reshape the input into a format compatible with LSTM layers
network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
# normalize input
network_input = network_input / float(n_vocab)

network_output = np_utils.to_categorical(network_output)

In [14]:
from keras.models import load_model, Model,Sequential
from keras.layers import Dense, Activation, Dropout, Input, LSTM, Reshape, Lambda, RepeatVector,GRU
from keras.initializers import glorot_uniform
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras import backend as K

In [15]:
from keras.callbacks import *

model = Sequential()
model.add(GRU(256,input_shape=(network_input.shape[1], network_input.shape[2]),return_sequences=True))
model.add(Dropout(0.3))
model.add(GRU(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(GRU(256))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(156))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

In [16]:
filepath = "mixed-weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"    

checkpoint = ModelCheckpoint(
    filepath, monitor='loss', 
    verbose=0,        
    save_best_only=True,        
    mode='min'
)    
callbacks_list = [checkpoint]     

model.fit(network_input, network_output, epochs=250, batch_size=512, callbacks=callbacks_list)

Epoch 1/250
4688/4688 [==============================] - 93s 20ms/step - loss: 5.1765
Epoch 2/250
4688/4688 [==============================] - 47s 10ms/step - loss: 3.4916
Epoch 3/250
4688/4688 [==============================] - 48s 10ms/step - loss: 3.4370
Epoch 4/250
4688/4688 [==============================] - 50s 11ms/step - loss: 3.4205
Epoch 5/250
4688/4688 [==============================] - 53s 11ms/step - loss: 3.4618
Epoch 6/250
4688/4688 [==============================] - 46s 10ms/step - loss: 3.4104
Epoch 7/250
4688/4688 [==============================] - 43s 9ms/step - loss: 3.4049
Epoch 8/250
4688/4688 [==============================] - 45s 10ms/step - loss: 3.3363
Epoch 9/250
4688/4688 [==============================] - 44s 9ms/step - loss: 3.2734
Epoch 10/250
4688/4688 [==============================] - 44s 9ms/step - loss: 3.2858
Epoch 11/250
4688/4688 [==============================] - 46s 10ms/step - loss: 3.1733
Epoch 12/250
4688/4688 [==============================]

4688/4688 [==============================] - 46s 10ms/step - loss: 0.2374
Epoch 96/250
4688/4688 [==============================] - 47s 10ms/step - loss: 0.2354
Epoch 97/250
4688/4688 [==============================] - 45s 10ms/step - loss: 0.2760
Epoch 98/250
4688/4688 [==============================] - 43s 9ms/step - loss: 0.1891
Epoch 99/250
4688/4688 [==============================] - 45s 10ms/step - loss: 0.2282
Epoch 100/250
4688/4688 [==============================] - 45s 10ms/step - loss: 0.2301
Epoch 101/250
4688/4688 [==============================] - 46s 10ms/step - loss: 0.2226
Epoch 102/250
4688/4688 [==============================] - 45s 10ms/step - loss: 0.1745
Epoch 103/250
4688/4688 [==============================] - 46s 10ms/step - loss: 0.2211
Epoch 104/250
4688/4688 [==============================] - 45s 10ms/step - loss: 0.1709
Epoch 105/250
4688/4688 [==============================] - 45s 10ms/step - loss: 0.1698
Epoch 106/250
4688/4688 [==========================

4688/4688 [==============================] - 45s 10ms/step - loss: 0.0742
Epoch 189/250
4688/4688 [==============================] - 46s 10ms/step - loss: 0.0699
Epoch 190/250
4688/4688 [==============================] - 45s 10ms/step - loss: 0.0725
Epoch 191/250
4688/4688 [==============================] - 53s 11ms/step - loss: 0.0682
Epoch 192/250
4688/4688 [==============================] - 49s 10ms/step - loss: 0.0794
Epoch 193/250
4688/4688 [==============================] - 46s 10ms/step - loss: 0.0592
Epoch 194/250
4688/4688 [==============================] - 44s 9ms/step - loss: 0.0816
Epoch 195/250
4688/4688 [==============================] - 47s 10ms/step - loss: 0.0567
Epoch 196/250
4688/4688 [==============================] - 45s 10ms/step - loss: 0.0823
Epoch 197/250
4688/4688 [==============================] - 44s 9ms/step - loss: 0.0749
Epoch 198/250
4688/4688 [==============================] - 44s 9ms/step - loss: 0.0675
Epoch 199/250
4688/4688 [========================

In [ ]:
model.fit(network_input, network_output, epochs=250, batch_size=64, callbacks=callbacks_list)

Epoch 1/250
765/765 [==============================] - 16s 21ms/step - loss: 0.9423
Epoch 2/250
765/765 [==============================] - 17s 23ms/step - loss: 0.9067
Epoch 3/250
765/765 [==============================] - 15s 20ms/step - loss: 0.8898
Epoch 4/250
765/765 [==============================] - 19s 24ms/step - loss: 0.9161
Epoch 5/250
765/765 [==============================] - 16s 21ms/step - loss: 0.7686
Epoch 6/250
765/765 [==============================] - 18s 23ms/step - loss: 0.8409
Epoch 7/250
765/765 [==============================] - 14s 18ms/step - loss: 0.7233
Epoch 8/250
765/765 [==============================] - 14s 18ms/step - loss: 0.7352
Epoch 9/250
765/765 [==============================] - 20s 26ms/step - loss: 0.6057
Epoch 10/250
765/765 [==============================] - 15s 20ms/step - loss: 0.6438
Epoch 11/250
765/765 [==============================] - 16s 22ms/step - loss: 0.6067
Epoch 12/250
765/765 [==============================] - 16s 20ms/step - lo

In [14]:
print(network_output.shape)
print(network_input.shape)

(3165, 143)
(3165, 100, 1)


In [17]:
from keras.models import model_from_json
model_json=model.to_json()
with open('mixed5.json','w') as json_file:
    json_file.write(model_json)
model.save_weights("mixed5.h5")

print("saved model to Disc")

saved model to Disc


In [ ]:
from keras.models import model_from_json
model_json=model.to_json()
with open('mixed3.json','w') as json_file:
    json_file.write(model_json)
model.save_weights("mixed3.h5")

print("saved model to Disc")

# To generate output

In [8]:
model = Sequential()
model.add(LSTM(
        256,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        return_sequences=True
    ))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')


# Load the weights to each node
model.load_weights('weights-improvement-86-1.6182-bigger.hdf5')

In [29]:
from keras.models import model_from_json
json_file=open('mixed4.json','r')
loaded_model_json=json_file.read()
json_file.close()
model=model_from_json(loaded_model_json)
model.load_weights("mixed4.h5")
print('loaded model from json')

loaded model from json


In [2]:
network_input=3165

In [30]:
import numpy
n_vocab=21
from music21 import converter, instrument, note, chord
import glob



start = numpy.random.randint(0, len(network_input)-1)

int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

pattern = network_input[start]
prediction_output = []
print(type(pattern))
# generate 500 notes
for note_index in range(500):
    prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
    prediction_input = prediction_input / float(n_vocab)

    prediction = model.predict(prediction_input, verbose=0)

    index = numpy.argmax(prediction)
    result = int_to_note[index]
    prediction_output.append(result)

    #pattern.append(index)
    pattern = numpy.append(pattern,index)
    pattern = pattern[1:len(pattern)]

<class 'numpy.ndarray'>


In [31]:
from music21 import converter,instrument
offset = 0
output_notes = []

# create note and chord objects based on the values generated by the model

for pattern in prediction_output:
    # pattern is a chord
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            #new_note.storedInstrument = instrument.Piano()
            new_note.storedInstrument = instrument.Violin()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    # pattern is a note
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        #new_note.storedInstrument = instrument.Piano()
        new_note.storedInstrument = instrument.AltoSaxophone()
        output_notes.append(new_note)

    # increase offset each iteration so that notes do not stack
    offset += 0.5

In [32]:
from music21 import stream

midi_stream = stream.Stream(output_notes)

midi_stream.write('midi', fp='mixed9.mid')

'mixed9.mid'

In [118]:
from music21 import converter,instrument # or import *
s = converter.parse('Bhairav1.mid')

for p in s.parts:
    p.insert(0, instrument.Violin())

s.write('midi', 'Bhairav2.mid')

'Bhairav2.mid'